## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-23-36-00 +0000,bbc,Flight from London to Cancun lands in US after...,https://www.bbc.com/news/articles/c98jqve2q7mo
1,2025-07-11-23-26-14 +0000,bbc,Trump surveys damage in Texas as search contin...,https://www.bbc.com/news/articles/cr5vlp0pmdzo
2,2025-07-11-23-05-58 +0000,bbc,Canada's Carney talked tough on Trump - now so...,https://www.bbc.com/news/articles/cvg81518yyyo
3,2025-07-11-22-56-32 +0000,bbc,"US has resumed military supplies to Ukraine, Z...",https://www.bbc.com/news/articles/crl04200dp4o
4,2025-07-11-22-56-21 +0000,nyt,"Maya Ruler’s Tomb Is Unearthed in Belize, With...",https://www.nytimes.com/2025/07/10/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,12
15,canada,6
107,tariff,5
106,latest,4
97,rate,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,35
32,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...,35
29,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,31
36,2025-07-11-13-13-32 +0000,cbc,RECAP | Trump's latest tariff threat sends tra...,https://www.cbc.ca/news/politics/livestory/rec...,29
2,2025-07-11-23-05-58 +0000,bbc,Canada's Carney talked tough on Trump - now so...,https://www.bbc.com/news/articles/cvg81518yyyo,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,35,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
12,19,2025-07-11-20-46-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
29,19,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
21,15,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
51,12,2025-07-11-02-39-47 +0000,bbc,Watch: Smoke billows from luxury yacht fire in...,https://www.bbc.com/news/videos/c4g85005q93o
28,12,2025-07-11-15-41-42 +0000,bbc,Gaza's largest functioning hospital facing dis...,https://www.bbc.com/news/articles/cdx5zeywgrgo
8,11,2025-07-11-22-10-00 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo
24,11,2025-07-11-16-24-19 +0000,bbc,Russians tell the BBC how they think the Ukrai...,https://www.bbc.com/news/videos/c1wp55wqq0lo
46,11,2025-07-11-08-14-13 +0000,bbc,'I created it myself': The boy behind the vira...,https://www.bbc.com/news/articles/czxe7ey59r0o
11,10,2025-07-11-21-11-54 +0000,bbc,US court rejects plea deal for '9/11 mastermin...,https://www.bbc.com/news/articles/c78nxqqp2leo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
